# Load libraries and data

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from treelib import Tree

from sklearn.datasets import load_boston, load_breast_cancer
from sklearn.tree import plot_tree, DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

In [2]:
data = load_boston()

X = data['data']
y = data['target']

feature_names = data['feature_names']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

# Decision tree

## Split method

`plot_tree` 함수로 트리를 그려보면 각 노드의 상태가 표기되어 있습니다. 세부 사항은 [포스트](https://ywkim92.github.io/machine_learning/feature_importance/#%EC%8B%9C%EA%B0%81%ED%99%94)에 기재해두었습니다. 다른 값들에 비해 유독 두 번째 줄 즉 feature와 threshold가 어째서 그렇게 결정되는지 이해하기 어려웠습니다. `random_state`가 바뀌어도 분기 기준은 바뀌지 않았으므로 무작위는 아니었습니다. 

먼저 분기 때마다 feature가 어떻게 정해지는지 찾아봤습니다. 검색 능력이 신통치 않아 적당한 레퍼런스를 발견하지 못했습니다. 다음엔 가설을 세우고 검증을 진행했습니다. 학습 데이터의 변수 중 분산이 가장 큰 (독립)변수? unique value 개수가 가장 많은 변수? (regression tree의 경우) 레이블과의 피어슨 상관계수가 가장 높은 변수? 모두 아니었습니다.

그러다 Feature importance를 공부하던 중 무릎을 쳤습니다. 핵심은 **불순도 감소량(impurity decrease)** 이었습니다. scikit-learn의 의사결정나무 모델에서 parameter `splitter = 'best'`(default)로 설정했을 경우, 불순도를 가장 크게 감소시키는 feature와 threshold를 탐색해 분기 기준을 결정합니다.

간단한 회귀 트리 모델을 생성하고 위 주장을 검증해보겠습니다. 앞서 분할한 train data를 학습합니다. 최상위 root node의 분기 기준은 아래와 같습니다.

In [3]:
model_dt = DecisionTreeRegressor(max_leaf_nodes=10, random_state=42, )
model_dt.fit(X_train, y_train)

print('Feature for splitting root node:', feature_names[model_dt.tree_.feature[0]])
print('Threshold for splitting root node:', model_dt.tree_.threshold[0])

Feature for splitting root node: RM
Threshold for splitting root node: 6.940999984741211


Best splitter를 찾기 위해 brute force 방법을 사용했습니다.
1. 독립변수 A를 선택한다.  
2. 학습 데이터에서 해당 독립변수의 unique values를 추출하고 내림차순으로 정렬한다.  
3. 모든 인접한 두 unique values 사이의 평균을 계산한다. 예를 들어, uniuqe values가 `[4, 3, 2, 1]`이라면, 계산 결과는 `[3.5, 2.5, 1.5]`이다.  
4. 3번에서 얻은 리스트의 각 원소를 threshold value로 두고 split했을 때 불순도 감소량을 관찰한다. 불순도를 가장 크게 감소시키는 원소를 독립변수 A의 threshold로 정한다.(불순도 감소량 산출식은 [지난 포스트](https://ywkim92.github.io/machine_learning/feature_importance/#feature-importance) 참조)  
5. 모든 독립변수에 대해 1~4번을 반복한다.  
6. 불순도 감소량이 가장 큰 독립변수와 그 threshold로 해당 노드의 샘플을 split한다.

위 방법에 따라 코딩하고 root node의 분기 기준을 도출합니다.

In [4]:
def best_splitter_reg(node_samples, feature_names, criterion):
    '''Find the best splitter for a decision tree regressor.
    
    node_samples: train data(pandas DataFrame) whose the last column is label and its column name is 'label'
    '''
    n = node_samples.shape[0] # the number of samples for training
    dic= dict()
    for col in feature_names:
        tr = node_samples[[col, 'label']]
        nt = tr.shape[0]
        it = tr['label'].var()
        values = np.unique(tr[col])
        if values.size==1:
            continue
        thresholds = np.array([values[i:i+2].mean() for i in range(values.size-1)])[::-1]

        if values.size <=10:
            iter_lim = thresholds.size
        else:
            iter_lim = thresholds.size//2

        i_max = 0
        th_tag = 0
        lim = 0
        for th in thresholds:
            # right node
            trr = tr[(tr[col]>th)]
            ntr = trr.shape[0]
            # calculate impurity for regression tree; 'mae' or 'mse'(default)
            if criterion == 'mae':
                ir = np.abs(trr['label'] - np.median(trr['label'])).mean()
            else:
                ir = trr['label'].var()

            # left node
            trl = tr[tr[col]<=th]
            ntl = trl.shape[0]
            if criterion == 'mae':
                il = np.abs(trl['label'] - np.median(trl['label'])).mean()
            else:
                il = trl['label'].var()

            # calculate weighted impurity decrease
            i = (nt / n) * ( it - (ntl / nt) * il - (ntr / nt)* ir )

            if i > i_max:
                i_max = i
                th_tag = th

            lim+=1
            if lim==iter_lim:
                break

        dic[col] = (i_max, th_tag)
    
    best_feature_ = max(dic, key = lambda x: dic[x][0])
    best_threshold_ = dic[best_feature_][1]
    return best_feature_, best_threshold_

모든 경우의 수에 대해 불순도 감소량을 산출하다 보니 scikit-learn 모델에 비해 시간이 너무 오래 걸렸습니다. unique value의 중앙값 부근에서는 불순도가 정체되는 양상이 확인되므로 unique values의 상위 50%에 대해서만 계산하게끔 처리했습니다. `iter_lim`을 정의하고 그 값을 넘으면 for문을 종료했습니다. best splitter를 찾는 알고리즘에 대해서는 추가 리서치할 계획입니다.

* 정의한 `best_splitter_reg` 함수로 도출한 분기 기준은 scikit-learn의 결과와 동일합니다.

In [5]:
train = pd.DataFrame(np.hstack((X_train, y_train.reshape(-1,1))), columns=feature_names.tolist()+['label'])

print('The best splitter of the root node(feature, threshold):',best_splitter_reg(train, feature_names, model_dt.criterion))

The best splitter of the root node(feature, threshold): ('RM', 6.941)


* root node에서 분기된 두 노드에 대해 검증하여도 scikit-learn과 같은 결과를 얻습니다.  
1. for node #1(splitted to left)

In [6]:
print('* Using scikit-learn attributes')
print('Feature for splitting the 1st node:', feature_names[model_dt.tree_.feature[1]])
print('Threshold for splitting the 1st node:', model_dt.tree_.threshold[1],)

print('\n* Using implementation')
train_node1 = train[train['RM']<=6.941]
print('The best splitter of the 1st node(feature, threshold):', best_splitter_reg(train_node1, feature_names, model_dt.criterion))

* Using scikit-learn attributes
Feature for splitting the 1st node: LSTAT
Threshold for splitting the 1st node: 14.400000095367432

* Using implementation
The best splitter of the 1st node(feature, threshold): ('LSTAT', 14.399999999999999)


2. for node #2(splitted to right)

In [7]:
print('* Using scikit-learn attributes')
print('Feature for splitting the 2nd node:', feature_names[model_dt.tree_.feature[2]])
print('Threshold for splitting the 2nd node:', model_dt.tree_.threshold[2],)

print('\n* Using implementation')
train_node2 = train[train['RM']>6.941]
print('The best splitter of the 1st node(feature, threshold):', best_splitter_reg(train_node2, feature_names, model_dt.criterion))

* Using scikit-learn attributes
Feature for splitting the 2nd node: RM
Threshold for splitting the 2nd node: 7.437000036239624

* Using implementation
The best splitter of the 1st node(feature, threshold): ('RM', 7.436999999999999)


* `criterion = 'mae'`으로 설정했을 때도 마찬가지입니다.

In [8]:
model_dt1 = DecisionTreeRegressor(max_leaf_nodes=10, random_state=42, criterion='mae')
model_dt1.fit(X_train, y_train)

print('when criterion = "mae"')
print('* Using scikit-learn attributes')
print('Feature for splitting root node:', feature_names[model_dt1.tree_.feature[0]])
print('Threshold for splitting root node:', model_dt1.tree_.threshold[0])

print('\n* Using implementation')
print('The best splitter of the root node(feature, threshold):',best_splitter_reg(train, feature_names, model_dt1.criterion))

when criterion = "mae"
* Using scikit-learn attributes
Feature for splitting root node: RM
Threshold for splitting root node: 6.797000169754028

* Using implementation
The best splitter of the root node(feature, threshold): ('RM', 6.797)


## How to assign Value for each node: 노드에 속하는 샘플에 부여되는 label 값

* 회귀: 해당 노드에 속하는 샘플들의 평균입니다. 각 노드에 부여되는 label 값을 산출하는 코드를 짜서 검증해봅니다.

In [9]:
def tree_value_reg(X_train, y_train, tree_regressor):
    result = []
    n_node = tree_regressor.tree_.node_count
    for i in range(n_node):
        value = y_train[tree_regressor.decision_path(X_train).toarray()[:, i]==1].mean()
        result.append(value)
    return np.array(result)

In [10]:
tree_values_sklearn = model_dt.tree_.value.flatten()
tree_values_imp = tree_value_reg(X_train, y_train, model_dt)
print('Sklean == my implementation:',np.allclose(tree_values_sklearn, tree_values_imp))

Sklean == my implementation: True


* 분류: 각 class를 index로 하며 해당 노드에 속하는 class별 샘플의 개수를 원소로 하는 list입니다. 이는 predict 단계에서 class probabilities를 계산할 때 사용됩니다. 예를 들어 이진 분류 모델에서 leaf node N의 value가 `[2, 8]`이라면, leaf node N에 최종 도달한 샘플이 class 0일 확률은 0.2, class 1일 확률은 0.8로 계산됩니다. threshold가 0.5라면 이 샘플은 class 1로 분류되겠지요.  
코드로 구현한 후 그 결과를 scikit-learn 결과와 비교해봅니다. 검증 결과 동일합니다.

In [11]:
def tree_value_clf(X_train, y_train, tree_clf):
    n_class = tree_clf.n_classes_
    result = np.array([]).reshape(-1, n_class)
    n_node = tree_clf.tree_.node_count
    
    for i in range(n_node):
        uniq, value = np.unique(y_train[tree_clf.decision_path(X_train).toarray()[:, i]==1], return_counts=True)
        if value.size == n_class:
            result = np.vstack((result, value))
        else:
            value_ = np.zeros(n_class)
            value_[np.array(uniq, dtype=int)] = value
            result = np.vstack((result, value_))
        
    return np.array(result)

검증에 사용할 이진 분류 데이터 샘플(유방암 데이터)과 의사결정나무 분류 모델을 불러옵니다. 검증 결과 동일합니다.

In [14]:
data_clf = load_breast_cancer()
X_clf = data_clf['data']
y_clf = data_clf['target']

model_clf = DecisionTreeClassifier(max_leaf_nodes=10, random_state=42)
model_clf.fit(X_clf, y_clf)

tree_values_sklearn = model_clf.tree_.value[:, 0, :]
tree_values_imp = tree_value_clf(X_clf, y_clf, model_clf)
print('Sklean == my implementation:',np.allclose(tree_values_sklearn, tree_values_imp))

Sklean == my implementation: True
